In [2]:
import numpy as np
import random as random
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
from scipy.spatial import distance
from scipy.stats import mode
from pyproj import Proj,transform
import seaborn as sns
import copy
from scipy import sparse
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy", towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
vgrid = Proj(init="world:bng")
#Calculate Euclidean distances using H

# Make yourself the 'All_data' file, this only needs to be run once ever
It is commented out for me, uncomment this if it's your first time

- CompleteData: data set given from EuFMD, length = 342
- ImputedData: data set imputed from Livestock map of the world, length = 85,151
- Data: data set of both, length = 85,493

In [3]:
# def dfLLtoEN(df):
#     """Returns easting, northing tuple
#     """
#     vlon36, vlat36 = transform(v84,v36,df["long"].values,df["lat"].values)
#     result = vgrid(vlon36,vlat36)

#     # Transform the output to a Dataframe
#     eastnorth = pd.DataFrame(index=df.index)
#     for i in result:
#         eastnorth["Easting"] = result[0]
#         eastnorth["Northing"] = result[1]

#     return round(eastnorth)

In [4]:
# Path = '../StudyGroup/'
# CompleteData=pd.read_csv(Path+"completeData2.csv",header = 0) 
# CompleteData = CompleteData.drop(columns = 'Unnamed: 0')

In [5]:
# CompleteData[['Easting', 'Northing']] = dfLLtoEN(CompleteData[['lat', 'long']])

In [6]:
# CompleteData = CompleteData.rename(index=str, columns={'X__1': "study"})

In [7]:
# CompleteData.columns.values

In [8]:
# ImputedData = pd.read_csv('imputed_farm.txt', header = 0, sep = '\t')
# ImputedData = ImputedData.rename(index=str, columns={'Unnamed: 0': "study"})

In [9]:
# ImputedData[['Easting', 'Northing']] = dfLLtoEN(ImputedData[['lat', 'long']])

In [10]:
# ImputedData['sr'] = ImputedData['ruminant']

In [11]:
# Data = pd.concat([CompleteData, ImputedData], ignore_index=True)

In [12]:
# xmax = np.max(Data['Easting'].values)
# xmin = np.min(Data['Easting'].values)
# ymax = np.max(Data['Northing'].values)
# ymin = np.min(Data['Northing'].values)
# sizex = xmax-xmin
# sizey = ymax-ymin
# Data['xcoord'] = Data['Easting'] - xmin
# Data['ycoord'] = Data['Northing'] - ymin

In [13]:
# Data.to_csv('All_data', sep='\t')

# Below, can be run once you have made 'All_data'

In [17]:
Data=pd.read_csv("All_data.txt",sep = '\t', header = 0)

In [18]:
Copy1 = pd.read_csv('All_data.txt', sep = '\t', header = 0)

In [19]:
copy2 = Copy1.values
true_cattle =  copy.deepcopy(copy2[:,3])
true_sheep =  copy.deepcopy(copy2[:,13])
cattle = copy2[:,3]
sheep = copy2[:,13]

In [20]:
#sheep
print('mean number sheep: \n', np.mean(sheep))
print('mode number: \n', mode(sheep)[0][0])
print('max, min: \n', np.max(sheep), np.min(sheep))

#cows
print('\n mean number cattle: \n', np.mean(cattle))
print('mode number: \n', mode(cattle)[0][0])
print('max, min: \n', np.max(cattle), np.min(cattle))

mean number sheep: 
 3.357553838566497
mode number: 
 0.0
max, min: 
 180.0 0.0

 mean number cattle: 
 6.714614499424626
mode number: 
 0
max, min: 
 132 0


In [21]:
xcoord = Data['xcoord'].values
ycoord = Data['ycoord'].values

In [22]:
joinedinput = np.column_stack((xcoord, ycoord))

In [27]:
N = len(Data)
N

6083

## Memory Issues

Cannot store a distance matrix containing between every farm (especially as it is not sparse)

In [12]:
equipment_list = Data['equipment'].values
shares_water_list = Data['water'].values
shares_grazing_list = Data['grazing'].values
shares_milk_list = Data['milk'].values
shares_vet_list = Data['vet'].values
contact_animal_list = Data['contact_animal'].values
contact_human_list = Data['contact_people'].values

## Create Sparse matrices

Individual matrices for:
- shares_milk_list
- shares_water_list
- shares_grazing_list
- shares_equipment_list
- shares_vet_list
- contact_animal_list (**no cap?**)
- contact_human_list (**no cap?**)

Initially start with a 10km cap, look at policies that would reduce this.

**Benchmark**: for `attributes_matrix(shares_water, 10, N= 85,493)` it takes 21570.181146383286 seconds

In [29]:
# This takes quite a while to run, sit back and have a cuppa
# once this has run, you won't need to run it again unless your kernel restarts

def attributes_matrix(arr, cap, n, name_str):
#     start = time.time()
    row_ind = []
    col_ind = []
    data_arr = []
    for i in range(n):
        for j in range(i+1):
            if i != j:
                dist = np.linalg.norm(joinedinput[i]-joinedinput[j])
                dist = dist/100000
                if dist <= cap/100:
                    if (arr[i] + arr[j] == 2):
                        row_ind.append(i)
                        col_ind.append(j)
                        data_arr.append(1)
#     end = time.time()
#     print(end-start)
    np.save('{}_rows'.format(name_str), row_ind)
    np.save('{}_cols'.format(name_str), col_ind)
    np.save('{}_data'.format(name_str), data_arr)
    return row_ind, col_ind, data_arr

# Example

In [28]:
row_ind = []
col_ind = []
data_arr = []
for i in range(N):
    for j in range(i+1):
        if i != j:
            dist = np.linalg.norm(joinedinput[i]-joinedinput[j])
            dist = dist/100000
            if dist <= 10/100:
                row_ind.append(i)
                col_ind.append(j)
                data_arr.append(1)
#     end = time.time()
#     print(end-start)
np.save('distance_rows', row_ind)
np.save('distance_cols', col_ind)
np.save('distance_data', data_arr)

In [34]:
row_ind[5000]
col_ind[5000]

173

In [38]:
# np.save('water_rows', rows)
# np.save('water_cols', cols)
# np.save('water_data', water)

In [43]:
rows = np.array(rows)
cols = np.array(cols)
water = np.array(water)

In [44]:
mat_coo = sparse.coo_matrix((water, (rows, cols)))

# Create a npz file for all attributes

In [117]:
names = ['shares_equipment', 'shares_water', 'shares_grazing', 'shares_milk', 'shares_vet', 'contact_animal', 'contact_human']

In [46]:
def sparse_matrix(name, sparse_type):
    row = np.load('{}_rows.npy'.format(name))
    col = np.load('{}_cols.npy'.format(name))
    data = np.load('{}_data.npy'.format(name))
    row_col = np.concatenate((row, col), axis = None) #do this because symmetric matrix (stored upper triangular)
    col_row = np.concatenate((col, row), axis = None)
    data = np.concatenate((data, data), axis = None)
    mat = sparse_type((data, (row_col, col_row)))
    sparse.save_npz('{}_sparse'.format(name), mat)
    return mat

In [ ]:
#first time running it
shares_equipment = sparse_matrix('shares_equipment', sparse.csc_matrix)
shares_water = sparse_matrix('shares_water', sparse.csc_matrix)
shares_grazing = sparse_matrix('shares_grazing', sparse.csc_matrix)
shares_milk = sparse_matrix('shares_milk', sparse.csc_matrix)
shares_vet = sparse_matrix('shares_vet', sparse.csc_matrix)
contact_animal = sparse_matrix('contact_animal', sparse.csc_matrix)
contact_human = sparse_matrix('contact_human', sparse.csc_matrix)

In [47]:
dist_mat = sparse_matrix('distance', sparse.csc_matrix)

In [50]:
import scipy.sparse as sp

In [51]:
sp.find(dist_mat<= 10/100)

/home/emma/.local/lib/python3.5/site-packages/scipy/sparse/compressed.py:274: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using <= is inefficient, try using > instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


KeyboardInterrupt: 